In [14]:
import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# Target classes in CIFAR-10: Cat (3), Dog (5), Ship (8), Truck (9)
allowed_labels = [3, 5, 8, 9]

In [15]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())


100%|██████████| 170M/170M [00:03<00:00, 50.6MB/s] 


In [21]:
plt.imshow(train_dataset[0][0].permute(1, 2, 0))
plt.show()

In [23]:
plt.plot([1,2,3], [1,2,3])
plt.show()